In [1]:
from __future__ import print_function
import librosa
import numpy as np
import os
import sys
import argparse

import tensorflow as tf

from keras.models import Sequential
from keras.layers import LSTM, TimeDistributed, Dense
from keras.optimizers import RMSprop, Adam, SGD
from keras import backend as K

from mirex2016_dataset import load_trainXY


Using TensorFlow backend.


In [2]:
def f1_score(y_true, y_pred):
    #Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))
    #If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
         return 0
    # How many selected items are relevant?
    precision = c1 / c2
    recall = c1 / c3
    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def recall(y_true, y_pred):
    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))
    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0
    recall = c1 / c3
    return recall

In [ ]:
'''
parser = argparse.ArgumentParser()
parser.add_argument('--datadir', type=str, default='./dataset/mirex_beat_tracking_2016/train/x431x128_y40x431/', help='train data path')
parser.add_argument('--logdir',  type=str, default='./dataset/log', help='Tensorboard log path')
parser.add_argument('--epochs',  type=int, default=200, help='the number of times to iterate over the training data')
parser.add_argument('-lr', type=lambda lr_str: [int(lr) for lr in lr_str.split(',')],
    default=[10, 1, 0.1, 0.01, 0.006, 0.003, 0.001, 0.0006, 0.0003, 0.0001, 0.00005, 0.00001], help='learning rate')
'''
#args = parser.parse_args()

In [3]:
#datadir = './dataset/mirex_beat_tracking_2016/train/x431x128_y40x431/'
#datadir = './dataset/mirex_beat_tracking_2016/train/x862x1025_y40x862/'
datadir = './x862x128_y862x1/'


In [4]:
trainXY_path = datadir
#tb_logdir = args.logdir
epochs = 500
learning_rates = [10, 1, 0.1, 0.01, 0.006, 0.003, 0.001, 0.0006, 0.0003, 0.0001, 0.00005, 0.00001]
loss_function = 'binary_crossentropy'


In [5]:
trainX, trainY, n_of_frames, n_of_freq_bins = load_trainXY(trainXY_path)
trainX.resize(trainX.shape[0], 1, n_of_freq_bins)
trainY.resize(trainY.shape[0], 1, 1)
#trainX.resize(n_of_frames*20*2, 1, n_of_freq_bins)
#trainY.resize(n_of_frames*20*2, 1, 1)

Load trainX.npy and trainY.npy (21, 128)


In [6]:
# reshape from mx1xn --> mxn
trainXP = trainX.reshape(trainX.shape[0],n_of_freq_bins)
trainYP = trainY.reshape(trainY.shape[0],1)
print('trainXP.shape: ' + str(trainXP.shape))
print('trainYP.shape: ' + str(trainYP.shape))

trainXP.shape: (17240, 128)
trainYP.shape: (17240, 1)


In [7]:
import pandas as pd
trainXPD  = np.diff(trainXP,axis=0)
trainYPD  = trainYP[1:]
trainXPDH = np.maximum(np.zeros(trainXPD.shape),trainXPD)
print(trainXPD.shape)
xx = pd.DataFrame(trainXP[0:4][:])
dd = pd.DataFrame(trainXPD[0:3][:])
hh = pd.DataFrame(trainXPDH[0:5][:])
display(xx)
print('\n')
display(dd)
print('\n')
display(hh)

# reshape from mxn --> mx1xn
trainXPDH3 = trainXPDH.reshape(trainXPDH.shape[0],1,trainXPDH.shape[1])
trainYPD3  = trainYPD.reshape(trainYPD.shape[0],1,trainYPD.shape[1])

(17239, 128)


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,...,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.0,-80.0,-80.0
1,-78.416825,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,...,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.0,-80.0,-80.0
2,-16.782757,-12.069625,-7.636879,-10.055699,-12.186193,-14.297897,-21.732388,-26.662192,-26.433747,-27.860949,...,-55.524676,-55.514602,-53.402921,-55.703926,-58.417543,-63.255391,-72.216577,-80.0,-80.0,-80.0
3,-3.696009,-6.775470,-12.168939,-16.854252,-18.960096,-24.425571,-27.194247,-27.391886,-28.151292,-35.199834,...,-37.081288,-42.817498,-44.738287,-43.906941,-43.328234,-54.360452,-54.851924,-80.0,-80.0,-80.0


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,1.583175,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,61.634068,67.930375,72.363121,69.944301,67.813807,65.702103,58.267612,53.337808,53.566253,52.139051,...,24.475324,24.485398,26.597079,24.296074,21.582457,16.744609,7.783423,0.0,0.0,0.0
2,13.086748,5.294155,-4.532059,-6.798553,-6.773904,-10.127674,-5.461859,-0.729694,-1.717545,-7.338885,...,18.443388,12.697104,8.664634,11.796985,15.089309,8.894939,17.364653,0.0,0.0,0.0


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,1.583175,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,61.634068,67.930375,72.363121,69.944301,67.813807,65.702103,58.267612,53.337808,53.566253,52.139051,...,24.475324,24.485398,26.597079,24.296074,21.582457,16.744609,7.783423,0.0,0.0,0.0
2,13.086748,5.294155,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,18.443388,12.697104,8.664634,11.796985,15.089309,8.894939,17.364653,0.0,0.0,0.0
3,0.000000,0.000000,0.739930,0.000000,0.000000,0.000000,0.000000,0.000000,1.397148,5.312010,...,0.000000,1.089796,0.000000,1.769560,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.804787,...,0.000000,1.297082,4.546216,0.000000,0.000000,26.797198,0.000000,0.0,0.0,0.0


In [8]:
model = Sequential()
#BATCH_SIZE  = 150
TIME_STEPS  = 1
INPUT_SIZE  = n_of_freq_bins
OUTPUT_SIZE = 1
CELL_SIZE   = 20
BATCH_START = 0
model.add(LSTM(
    units = CELL_SIZE,
    #batch_input_shape=(BATCH_SIZE, TIME_STEPS, INPUT_SIZE),       # Or: input_dim=INPUT_SIZE, input_length=TIME_STEPS,
    input_shape = (TIME_STEPS,INPUT_SIZE),
    #output_dim=CELL_SIZE,
    return_sequences=True,      # True: output at all steps. False: output as last step.
    stateful=False,              # True: the final state of batch1 is feed into the initial state of batch2
))
#model.add(LSTM(20))
# add output layer
model.add(TimeDistributed(Dense(OUTPUT_SIZE)))
#model.add(Dense(OUTPUT_SIZE))
lr = 0.000001
#rmsprop = RMSprop(lr=lr, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss=loss_function, optimizer='adam', metrics=['accuracy',recall])

In [ ]:
X = trainXPDH3
Y = trainYPD3
model.fit(X, Y, epochs=500, batch_size=431)

Epoch 1/500
17239/17239 [==============================] - 0s - loss: 2.5738 - acc: 0.4911 - recall: 0.3913     
Epoch 2/500
17239/17239 [==============================] - 0s - loss: 1.0486 - acc: 0.5064 - recall: 0.4273     
Epoch 3/500
17239/17239 [==============================] - 0s - loss: 0.8499 - acc: 0.5172 - recall: 0.5036     
Epoch 4/500
17239/17239 [==============================] - 0s - loss: 0.7906 - acc: 0.5124 - recall: 0.4467     
Epoch 5/500
17239/17239 [==============================] - 0s - loss: 0.7580 - acc: 0.5272 - recall: 0.5407     
Epoch 6/500
17239/17239 [==============================] - 0s - loss: 0.7383 - acc: 0.5352 - recall: 0.5811     
Epoch 7/500
17239/17239 [==============================] - 0s - loss: 0.7248 - acc: 0.5423 - recall: 0.5981     
Epoch 8/500
17239/17239 [==============================] - 0s - loss: 0.7172 - acc: 0.5482 - recall: 0.6109     
Epoch 9/500
17239/17239 [==============================] - 0s - loss: 0.7108 - acc: 0.5535 - rec

17239/17239 [==============================] - 0s - loss: 0.5188 - acc: 0.7196 - recall: 0.7645     
Epoch 73/500
17239/17239 [==============================] - 0s - loss: 0.5158 - acc: 0.7224 - recall: 0.7598     
Epoch 74/500
17239/17239 [==============================] - 0s - loss: 0.5123 - acc: 0.7257 - recall: 0.7711     
Epoch 75/500
17239/17239 [==============================] - 0s - loss: 0.5103 - acc: 0.7252 - recall: 0.7707     
Epoch 76/500
17239/17239 [==============================] - 0s - loss: 0.5091 - acc: 0.7304 - recall: 0.7754     
Epoch 77/500
17239/17239 [==============================] - 0s - loss: 0.5041 - acc: 0.7312 - recall: 0.7809     
Epoch 78/500
17239/17239 [==============================] - 0s - loss: 0.5020 - acc: 0.7300 - recall: 0.7767     
Epoch 79/500
17239/17239 [==============================] - 0s - loss: 0.4986 - acc: 0.7336 - recall: 0.7788     
Epoch 80/500
17239/17239 [==============================] - 0s - loss: 0.4957 - acc: 0.7354 - recall:

17239/17239 [==============================] - 0s - loss: 0.3597 - acc: 0.7833 - recall: 0.8414     
Epoch 144/500
17239/17239 [==============================] - 0s - loss: 0.3569 - acc: 0.7820 - recall: 0.8472     
Epoch 145/500
17239/17239 [==============================] - 0s - loss: 0.3526 - acc: 0.7834 - recall: 0.8492     
Epoch 146/500
17239/17239 [==============================] - 0s - loss: 0.3505 - acc: 0.7846 - recall: 0.8504     
Epoch 147/500
17239/17239 [==============================] - 0s - loss: 0.3499 - acc: 0.7833 - recall: 0.8500     
Epoch 148/500
17239/17239 [==============================] - 0s - loss: 0.3474 - acc: 0.7847 - recall: 0.8516     
Epoch 149/500
17239/17239 [==============================] - 0s - loss: 0.3475 - acc: 0.7837 - recall: 0.8489     
Epoch 150/500
17239/17239 [==============================] - 0s - loss: 0.3430 - acc: 0.7818 - recall: 0.8529     
Epoch 151/500
17239/17239 [==============================] - 0s - loss: 0.3418 - acc: 0.7848 -

17239/17239 [==============================] - 0s - loss: 0.2806 - acc: 0.7709 - recall: 0.8846     
Epoch 215/500
17239/17239 [==============================] - 0s - loss: 0.2724 - acc: 0.7713 - recall: 0.8915     
Epoch 216/500
17239/17239 [==============================] - 0s - loss: 0.2724 - acc: 0.7706 - recall: 0.8930     
Epoch 217/500
17239/17239 [==============================] - 0s - loss: 0.2722 - acc: 0.7690 - recall: 0.8898     
Epoch 218/500
17239/17239 [==============================] - 0s - loss: 0.2706 - acc: 0.7705 - recall: 0.8936     
Epoch 219/500
17239/17239 [==============================] - 0s - loss: 0.2682 - acc: 0.7706 - recall: 0.8958     
Epoch 220/500
17239/17239 [==============================] - 0s - loss: 0.2661 - acc: 0.7721 - recall: 0.8953     
Epoch 221/500
17239/17239 [==============================] - 0s - loss: 0.2669 - acc: 0.7709 - recall: 0.8944     
Epoch 222/500
17239/17239 [==============================] - 0s - loss: 0.2657 - acc: 0.7693 -

17239/17239 [==============================] - 0s - loss: 0.2593 - acc: 0.7594 - recall: 0.9077     
Epoch 286/500
17239/17239 [==============================] - 0s - loss: 0.2611 - acc: 0.7590 - recall: 0.9044     
Epoch 287/500
17239/17239 [==============================] - 0s - loss: 0.2657 - acc: 0.7572 - recall: 0.9025     
Epoch 288/500
17239/17239 [==============================] - 0s - loss: 0.2590 - acc: 0.7577 - recall: 0.9075     
Epoch 289/500
17239/17239 [==============================] - 0s - loss: 0.2564 - acc: 0.7589 - recall: 0.9087     
Epoch 290/500
17239/17239 [==============================] - 0s - loss: 0.2544 - acc: 0.7582 - recall: 0.9098     
Epoch 291/500
17239/17239 [==============================] - 0s - loss: 0.2529 - acc: 0.7562 - recall: 0.9105     
Epoch 292/500
17239/17239 [==============================] - 0s - loss: 0.2513 - acc: 0.7591 - recall: 0.9108     
Epoch 293/500
17239/17239 [==============================] - 0s - loss: 0.2504 - acc: 0.7582 -

17239/17239 [==============================] - 0s - loss: 0.2082 - acc: 0.7266 - recall: 0.9269     
Epoch 357/500
17239/17239 [==============================] - 0s - loss: 0.2074 - acc: 0.7248 - recall: 0.9277     
Epoch 358/500
17239/17239 [==============================] - 0s - loss: 0.2064 - acc: 0.7259 - recall: 0.9255     
Epoch 359/500
17239/17239 [==============================] - 0s - loss: 0.2112 - acc: 0.7246 - recall: 0.9139     
Epoch 360/500
17239/17239 [==============================] - 0s - loss: 0.2307 - acc: 0.7223 - recall: 0.9076     
Epoch 361/500
17239/17239 [==============================] - 0s - loss: 0.2281 - acc: 0.7191 - recall: 0.9176     
Epoch 362/500
17239/17239 [==============================] - 0s - loss: 0.2243 - acc: 0.7178 - recall: 0.9182     
Epoch 363/500
17239/17239 [==============================] - 0s - loss: 0.2178 - acc: 0.7169 - recall: 0.9225     
Epoch 364/500
17239/17239 [==============================] - 0s - loss: 0.2089 - acc: 0.7210 -

17239/17239 [==============================] - 0s - loss: 0.8288 - acc: 0.5901 - recall: 0.8400     
Epoch 428/500
17239/17239 [==============================] - 0s - loss: 0.6836 - acc: 0.6237 - recall: 0.8552     
Epoch 429/500
17239/17239 [==============================] - 0s - loss: 0.6763 - acc: 0.6316 - recall: 0.8541     
Epoch 430/500
17239/17239 [==============================] - 0s - loss: 0.5678 - acc: 0.6431 - recall: 0.8615     
Epoch 431/500
17239/17239 [==============================] - 0s - loss: 0.5660 - acc: 0.6456 - recall: 0.8350     
Epoch 432/500
17239/17239 [==============================] - 0s - loss: 0.5166 - acc: 0.6531 - recall: 0.8753     
Epoch 433/500
17239/17239 [==============================] - 0s - loss: 0.4525 - acc: 0.6538 - recall: 0.8735     
Epoch 434/500
17239/17239 [==============================] - 0s - loss: 0.4812 - acc: 0.6425 - recall: 0.8738     
Epoch 435/500
17239/17239 [==============================] - 0s - loss: 0.5902 - acc: 0.6328 -

In [ ]:
'''
# tensorboard representation
tb_logpath = tb_logdir + '/%s_rmsprop_lr_%f' % (loss_function, lr)
tb = tf.contrib.keras.callbacks.TensorBoard(log_dir=tb_logpath, histogram_freq=0, write_graph=True, write_images=True)
'''